In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eclipse-train-test2/train2.txt
/kaggle/input/eclipse-train-test2/test2.txt
/kaggle/input/eclipse-train-test1/test1.txt
/kaggle/input/eclipse-train-test1/train1.txt
/kaggle/input/eclipse-train-test5/train5_dt5.txt
/kaggle/input/eclipse-train-test5/test5_dt5.txt
/kaggle/input/eclipse-preprocessed5new/eclipse_preprocessed_os.csv


In [22]:
import pandas as pd
import numpy as np
from random import seed
from random import randint
import random
import string
import re
import matplotlib.pyplot as plt

In [23]:
import seaborn as sns
import transformers

import nltk


from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

plt.style.use('seaborn')

In [24]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
from tensorflow.keras.layers import Dense, Input, Reshape, Conv1D, Conv2D, BatchNormalization, MaxPooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [26]:
seed(1)
stop_words = stopwords.words('english')

In [27]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.6.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [28]:
train_indices = []
test_indices = []

with open('../input/eclipse-train-test5/train5_dt5.txt') as f:
    train_indices = f.readlines()

with open('../input/eclipse-train-test5/test5_dt5.txt') as f:
    test_indices = f.readlines()

In [29]:
train_indices = [int(tr.split('\n')[0]) for tr in train_indices]

In [30]:
len(train_indices)

192911

In [31]:
test_indices  = [int(te.split('\n')[0]) for te in test_indices]

In [32]:
len(test_indices)

48227

In [33]:
PATH_CSV_TRAIN = '../input/eclipse-preprocessed5new/eclipse_preprocessed_os.csv'
# PATH_CSV_TEST = '../input/nlpgettingstarted/test.csv'
# PATH_CSV_SUBMISSION = '../input/nlpgettingstarted/sample_submission.csv'

dataset = pd.read_csv(PATH_CSV_TRAIN)
# dataf_test = pd.read_csv(PATH_CSV_TEST)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
dataset.dropna(subset = ["description1"], inplace=True)
dataset.dropna(subset = ["description2"], inplace=True)

In [35]:
def reset_column_names():
  dataset.drop('description1', axis=1, inplace=True)
  dataset.drop('short_desc1', axis=1, inplace=True)
  dataset.drop('description2', axis=1, inplace=True)
  dataset.drop('short_desc2', axis=1, inplace=True)

  dataset.rename(columns={'description1_clean':'description1','short_desc1_clean':'short_desc1','description2_clean':'description2','short_desc2_clean':'short_desc2'}, inplace=True)

In [36]:
def clean_text(text):
    #Remove emojis and special chars
    clean=text
    #reg = re.compile('\\.+?(?=\B|$)')
    #clean = text.apply(lambda r: re.sub(reg, string=r, repl=''))
    #reg = re.compile('\x89Û_')
    #clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
#     reg = re.compile('\&amp')
#     clean = clean.apply(lambda r: re.sub(reg, string=r, repl='&'))
#     reg = re.compile('\\n')
#     clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))

    #Remove hashtag symbol (#)
    #clean = clean.apply(lambda r: r.replace('#', ''))

    #Remove user names
#     reg = re.compile('@[a-zA-Z0-9\_]+')
#     clean = clean.apply(lambda r: re.sub(reg, string=r, repl='@'))

    #Remove URLs
#     reg = re.compile(r"https?:\/\/[A-Za-z0-9.\/?&#+*+-=_%]+")
#     clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    
#     reg = re.compile(r"""
#                [,.;:@#?!&$*+-=_%<>\/\[\]\(\)\{\}\"\'\n]+  # Accept one or more copies of punctuation
#                \ *           # plus zero or more copies of a space,
#                """)
#     clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    
    reg = re.compile(r" +")
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    
#     reg = re.compile(r"[d|D]escription")
#     clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    

    #Lowercase
    clean = clean.apply(lambda r: r.lower())
    return clean

In [37]:
dataset['description1_clean'] = clean_text(dataset['description1'])
dataset['short_desc1_clean'] = clean_text(dataset['short_desc1'])
dataset['description2_clean'] = clean_text(dataset['description2'])
dataset['short_desc2_clean'] = clean_text(dataset['short_desc2'])
dataset.head()

,Unnamed: 0,bug_id,description1,short_desc1,dup_id,resolution,product,component,description2,short_desc2,duplicate,description1_clean,short_desc1_clean,description2_clean,short_desc2_clean
0,59556,62303,There should be some global dialog or preferen...,[Keybindings] Provide a list of all keybindings,37439,DUPLICATE,Platform,UI,35722 it would be very useful it if were po...,35722 [KeyBindings] preference page: add a ...,1,there should be some global dialog or preferen...,[keybindings] provide a list of all keybindings,35722 it would be very useful it if were possi...,35722 [keybindings] preference page: add a way...
1,25380,26824,"When class X has a private field F, and code i...",New auto-suggestion; add 'generate getXxx' whe...,28068,DUPLICATE,JDT,UI,26619 I have two classes:\nA - with private...,"26619 [quick fix] Greate getter for ""Field ...",1,"when class x has a private field f, and code i...",new auto-suggestion; add 'generate getxxx' whe...,26619 i have two classes:\na - with private fi...,"26619 [quick fix] greate getter for ""field is ..."
2,84739,89233,Need to complete a WSDL model core API accordi...,Need to define WSDL Core API,89291,DUPLICATE,WTP Webservices,wst.wsdl,84804 Need to complete a WSDL model core AP...,84804 Need to define WSDL Core API,1,need to complete a wsdl model core api accordi...,need to define wsdl core api,84804 need to complete a wsdl model core api a...,84804 need to define wsdl core api
3,145562,155467,Build: RSA-M20060802-0800\n\nSteps:\n1) Wind...,No error message displays when choose an inval...,155442,DUPLICATE,EPF,Tool,145537 Build: RSA-M20060802-0800\n\nSteps:\...,145537 No error message displays when choos...,1,build: rsa-m20060802-0800\n\nsteps:\n1) window...,no error message displays when choose an inval...,145537 build: rsa-m20060802-0800\n\nsteps:\n1)...,145537 no error message displays when choose a...
4,186715,201768,Build ID: I20070625-1500\n\nSteps To Reproduce...,Should be able to middle click on editor/ view...,156792,DUPLICATE,Platform,UI,146782 It would be ui-friendly if the tabs ...,146782 [EditorMgmt] Close a Tab with the mi...,1,build id: i20070625-1500\n\nsteps to reproduce...,should be able to middle click on editor/ view...,146782 it would be ui-friendly if the tabs cou...,146782 [editormgmt] close a tab with the middl...


In [38]:
reset_column_names()

In [39]:
dataset

,Unnamed: 0,bug_id,dup_id,resolution,product,component,duplicate,description1,short_desc1,description2,short_desc2
0,59556,62303,37439,DUPLICATE,Platform,UI,1,there should be some global dialog or preferen...,[keybindings] provide a list of all keybindings,35722 it would be very useful it if were possi...,35722 [keybindings] preference page: add a way...
1,25380,26824,28068,DUPLICATE,JDT,UI,1,"when class x has a private field f, and code i...",new auto-suggestion; add 'generate getxxx' whe...,26619 i have two classes:\na - with private fi...,"26619 [quick fix] greate getter for ""field is ..."
2,84739,89233,89291,DUPLICATE,WTP Webservices,wst.wsdl,1,need to complete a wsdl model core api accordi...,need to define wsdl core api,84804 need to complete a wsdl model core api a...,84804 need to define wsdl core api
3,145562,155467,155442,DUPLICATE,EPF,Tool,1,build: rsa-m20060802-0800\n\nsteps:\n1) window...,no error message displays when choose an inval...,145537 build: rsa-m20060802-0800\n\nsteps:\n1)...,145537 no error message displays when choose a...
4,186715,201768,156792,DUPLICATE,Platform,UI,1,build id: i20070625-1500\n\nsteps to reproduce...,should be able to middle click on editor/ view...,146782 it would be ui-friendly if the tabs cou...,146782 [editormgmt] close a tab with the middl...
...,...,...,...,...,...,...,...,...,...,...,...
241158,189743,205251,[],FIXED,Platform,CVS,0,build id: m20060921-0945\n\nsteps to reproduce...,cvs username disallows full kerberos names,the current tptp source locator only searches ...,need non-java centric extensible source locator
241159,16340,17624,[],NDUPLICATE,Platform,SWT,0,f1 linux motif \n\nmnemonics do not work in ec...,mnemonics do not work,build identifier: 4.7.0_201004221916\n\nprevio...,execution statistics view is populated even wh...
241160,285393,320100,[],FIXED,e4,UI,0,1. open a fresh e4 workbench\n2. leave the pac...,package explorer unexpectedly added to debug p...,"""content assist"" did not complete normally. pl...","""content assist"" did not complete normally."
241161,237512,261239,[],INVALID,Platform,SWT,0,created attachment 122704\nimage of the proble...,swt styled text corrupts screen in arabic on m...,when comparing files which contain cvs keyword...,option to ignore cvs keywords


In [40]:
for index, row in dataset.iterrows():
    desc = row['description1'] + row['description2']
    dataset.loc[index, 'description'] = desc

In [41]:
from transformers import TFXLNetModel, XLNetTokenizer, XLNetConfig

In [42]:
xlnet_model = 'xlnet-base-cased'
xlnet_tokenizer = XLNetTokenizer.from_pretrained(xlnet_model)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [43]:
def create_xlnet(mname):
    """ Creates the model. It is composed of the XLNet main block and then
    a classification head its added
    """
    # Define token ids as inputs
    word_inputs = tf.keras.Input(shape=(160,), name='word_inputs', dtype='int32')

    # Call XLNet model
    xlnet = TFXLNetModel.from_pretrained(mname)
    xlnet_encodings = xlnet(word_inputs)[0]

    # CLASSIFICATION HEAD 
    # Collect last step from last hidden state (CLS)
    doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
    # Apply dropout for regularization
    doc_encoding = tf.keras.layers.Dropout(.1)(doc_encoding)
    # Final output 
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)

    # Compile model
    model = tf.keras.Model(inputs=[word_inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

In [44]:
def get_inputs(tweets, tokenizer, max_len=160):
    """ Gets tensors from text using the tokenizer provided"""
    inps = [tokenizer.encode_plus(t, max_length=max_len, pad_to_max_length=True, add_special_tokens=True) for t in tweets]
    inp_tok = np.array([a['input_ids'] for a in inps])
    ids = np.array([a['attention_mask'] for a in inps])
    segments = np.array([a['token_type_ids'] for a in inps])
    return inp_tok, ids, segments

In [45]:
X = dataset['description']
Y = np.array(dataset.duplicate.values, dtype='int')

In [46]:
# X.iloc[train_indices]

In [47]:
# Y[train_indices]

In [48]:
inp_tok, ids, segments = get_inputs(X.iloc[train_indices], xlnet_tokenizer)
y_train = np.array(Y[train_indices], dtype='int')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [49]:
xlnet = create_xlnet(xlnet_model)
xlnet.summary()
# Fit the model  
hist = xlnet.fit(x=inp_tok, y=y_train, epochs=5, batch_size=40)

Downloading:   0%|          | 0.00/539M [00:00<?, ?B/s]

2022-03-07 04:25:01.065633: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-07 04:25:01.067803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 04:25:01.068639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 04:25:01.069329: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_inputs (InputLayer)     [(None, 160)]             0         
_________________________________________________________________
tfxl_net_model (TFXLNetModel TFXLNetModelOutput(last_h 116718336 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 1, 768)            0         
_________________________________________________________________
tf.compat.v1.squeeze (TFOpLa (None, 768)               0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 768)               0         
_________________________________________________________________
outputs (Dense)              (None, 1)                 769       
Total params: 116,719,105
Trainable params: 116,719,105
Non-trainable params: 0
_______________________________________________

2022-03-07 04:25:15.808258: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
4823/4823 [==============================] - 4795s 992ms/step - loss: 0.2758 - accuracy: 0.8538 - precision: 0.8308 - recall: 0.7053
Epoch 2/5
4823/4823 [==============================] - 4783s 992ms/step - loss: 0.2134 - accuracy: 0.8891 - precision: 0.8486 - recall: 0.8122
Epoch 3/5
4823/4823 [==============================] - 4784s 992ms/step - loss: 0.1420 - accuracy: 0.9369 - precision: 0.9050 - recall: 0.9057
Epoch 4/5
4823/4823 [==============================] - 4783s 992ms/step - loss: 0.0860 - accuracy: 0.9654 - precision: 0.9479 - recall: 0.9485
Epoch 5/5
4823/4823 [==============================] - 4784s 992ms/step - loss: 0.0577 - accuracy: 0.9782 - precision: 0.9662 - recall: 0.9684


In [50]:
xlnet.save("eclipse_xlnet")

2022-03-07 11:04:15.185121: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [51]:
test_inp_tok, test_ids, test_segments = get_inputs(X.iloc[test_indices], xlnet_tokenizer)
y_test = np.array(Y[test_indices], dtype='int')

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [52]:
# Evaluate the model
scores = xlnet.evaluate(test_inp_tok,y_test, verbose=0)
f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
print("%s: %.2f%%" % (xlnet.metrics_names[1], scores[1]*100))
print("%s: %.2f%%" % (xlnet.metrics_names[2], scores[2]*100))
print("%s: %.2f%%" % (xlnet.metrics_names[3], scores[3]*100))
print("f1 score: ", (f1score))

accuracy: 96.02%
precision: 90.85%
recall: 97.93%
f1 score:  94.2585176074182


In [53]:
# X = dataset['description']
# Y = np.array(dataset.duplicate.values, dtype='int')
# # X = dataset['description']
# # Y = dataset['duplicate']

# # kfold = StratifiedKFold()
# cvscores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []
# inp_tok, ids, segments = []
# y_train = []
# test_inp_tok, test_ids, test_segments = []
# y_test = []
    
# for train in train_indices:
# #     print(X.iloc[train].shape)
# #     print(X.iloc[train])
# #     print(Y[train])

#     # Prepare data
#     inp_tok, ids, segments = get_inputs(X.iloc[train], xlnet_tokenizer)
#     y_train = np.array(Y[train], dtype='int')
# #     print(inp_tok, ids, segments)
# #     print(y_train)
    
# xlnet = create_xlnet(xlnet_model)
# xlnet.summary()
# # Fit the model  
# hist = xlnet.fit(x=inp_tok, y=y_train, epochs=2, batch_size=30)
    
# for test in test_indices:
#     test_inp_tok, test_ids, test_segments = get_inputs(X.iloc[test], xlnet_tokenizer)
#     y_test = np.array(Y[test], dtype='int')

#     # Evaluate the model
#     scores = xlnet.evaluate(test_inp_tok,y_test, verbose=0)
#     f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
#     print("%s: %.2f%%" % (xlnet.metrics_names[1], scores[1]*100))
#     print("%s: %.2f%%" % (xlnet.metrics_names[2], scores[2]*100))
#     print("%s: %.2f%%" % (xlnet.metrics_names[3], scores[3]*100))
#     print("f1 score: ", (f1score))
#     cvscores.append(scores[1] * 100)
#     precision_scores.append(scores[2] * 100)
#     recall_scores.append(scores[3] * 100)
#     f1_scores.append(f1score)

# print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
# print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision_scores), np.std(precision_scores)))
# print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall_scores), np.std(recall_scores)))
# print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))

In [54]:
# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4, min_delta=0.02, restore_best_weights=True),
#     tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=1e-6, patience=2, verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-6)
# ]

In [55]:
# preds = xlnet.predict(inp_tok, verbose=True)

In [56]:
# plot_metrics(preds, y_test)